In [1]:
#Import functions and load data
import os
os.chdir("../src")
import tensorflow as tf
import numpy as np
from dataloader import qm9_parse, qm9_fetch
import dmol
import torch
import torch.nn as nn
qm9_records = qm9_fetch()
data = qm9_parse(qm9_records)


2023-11-27 15:42:05.578259: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 15:42:06.341864: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-27 15:42:06.341929: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-27 15:42:09.061543: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Found existing record file, delete if you want to re-fetch


2023-11-27 15:42:16.920545: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-11-27 15:42:16.920630: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-27 15:42:16.920671: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Katrines): /proc/driver/nvidia/version does not exist
2023-11-27 15:42:16.921280: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
shuffled_data = data.shuffle(7000, reshuffle_each_iteration=False)
test_set = shuffled_data.take(1000)
valid_set = shuffled_data.skip(1000).take(1000)
train_set = shuffled_data.skip(2000).take(5000)


In [3]:
import torch
import torch.nn as nn

def convert_record(d, atom_types=100, embedding_dim=128):
    # break up record
    (e, x), y = d

    # Convert to PyTorch tensors
    e = torch.tensor(e.numpy())
    x = torch.tensor(x.numpy())
    r = x[:, :3]

    # Assuming atom indices start from 1
    e = e - 1
    e = torch.clamp(e, 0, atom_types - 1)  # Ensure indices are within valid range

    # Embedding
    embedding_layer = nn.Embedding(num_embeddings=atom_types, embedding_dim=embedding_dim)
    s = embedding_layer(e)

    return (s, r), y.numpy()[13]  # Select attribute at index 13


#
def x2e(x, cutoff_distance=5.0):
    """convert xyz coordinates to pairwise distance with a cutoff distance"""
   # Calculate pairwise distances
   # this calculates the norm
    #r0 = (x- x[:, None, :]) #TODO: RIJ
    r2 = torch.sqrt(((x - x[:, None, :])**2).sum(dim=-1))

    # Create a mask for distances less than cutoff_distance
    mask = (r2>0) & (r2 <= cutoff_distance)

    # Use the mask to set values in the tensor
    r_ij = torch.where(mask, r2, torch.zeros_like(r2))

    # Generate edge index matrix
    #edge_index = torch.nonzero(mask, as_tuple=False)

    #edge_mask = (r2 > 0) & (r2 < cutoff_distance)
    edge_indices = mask.nonzero(as_tuple=True)
    edge_index = torch.stack(edge_indices)
    #edge_index = edge_index.resize_(2,len(mask))

    return r_ij, edge_index


In [6]:
# for d in test_set:
#     (s, r_ij), y_raw = convert_record(d)


# Assuming test_set is a list of data points
s_j_samlet = []
r_ij_samlet = []
y_raw_samlet =[]

for d in train_set:
    (s_j, r_ij), y_raw = convert_record(d)
    s_j_samlet.append(s_j)

    r2, edge_index = x2e(r_ij)
    r_ij_samlet.append(r2)
    y_raw_samlet.append(y_raw)


In [5]:
# TODO Kat tror at vi her skal zip


tensor([[ 0.2813,  0.6982,  1.3782,  ...,  2.2824,  0.1095,  0.0165],
        [-0.2957,  2.3361,  0.7956,  ..., -0.2223, -0.0924,  0.0515],
        [ 0.2813,  0.6982,  1.3782,  ...,  2.2824,  0.1095,  0.0165],
        ...,
        [ 0.5694, -0.2632, -0.2392,  ...,  0.1870,  1.4736, -0.6734],
        [ 0.5694, -0.2632, -0.2392,  ...,  0.1870,  1.4736, -0.6734],
        [ 0.5694, -0.2632, -0.2392,  ...,  0.1870,  1.4736, -0.6734]],
       grad_fn=<EmbeddingBackward0>)

In [7]:
#Normalize y values first and transform after prediction
ys = [convert_record(d)[1] for d in train_set]
train_ym = np.mean(ys)
train_ys = np.std(ys)
def transform_label(y):
    return (y - train_ym) / train_ys
def transform_prediction(y):
    return y * train_ys + train_ym


# Message block

In [8]:
import math

class CosineCutoff(torch.nn.Module):

    def __init__(self, cutoff=5.0):
        super(CosineCutoff, self).__init__()
        #self.register_buffer("cutoff", torch.FloatTensor([cutoff]))
        self.cutoff = cutoff

    def forward(self, distances):
        """Compute cutoff.

        Args:
            distances (torch.Tensor): values of interatomic distances.

        Returns:
            torch.Tensor: values of cutoff function.

        """
        # Compute values of cutoff function
        cutoffs = 0.5 * (torch.cos(distances * np.pi / self.cutoff) + 1.0)
        # Remove contributions beyond the cutoff radius
        cutoffs *= (distances < self.cutoff).float()
        return cutoffs

class BesselBasis(torch.nn.Module):
    """
    Sine for radial basis expansion with coulomb decay. (0th order Bessel from DimeNet)
    """

    def __init__(self, cutoff=5.0, n_rbf=20):
        """
        Args:
            cutoff: radial cutoff
            n_rbf: number of basis functions.
        """
        super(BesselBasis, self).__init__()
        # compute offset and width of Gaussian functions
        freqs = torch.arange(1, n_rbf + 1) * math.pi / cutoff
        self.register_buffer("freqs", freqs)

    def forward(self, inputs):
        inputs = torch.norm(inputs, p=2, dim=1)
        a = self.freqs
        ax = torch.outer(inputs,a)
        sinax = torch.sin(ax)

        norm = torch.where(inputs == 0, torch.tensor(1.0, device=inputs.device), inputs)
        y = sinax / norm[:,None]

        return y


In [9]:
# !pip install torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [10]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import MessagePassing, radius_graph
from torch_geometric.utils import add_self_loops, degree

import ase
import torch.nn as nn
import torch.nn.functional as Func
from torch.nn import Embedding, Sequential, Linear, ModuleList, Module
import numpy as np
from torch import linalg as LA
import math

from torch_geometric.data import Data


In [35]:
class MessagePassPaiNN(MessagePassing):
    def __init__(self, num_feat, out_channels, num_nodes, cut_off=5.0, n_rbf=20):
        super(MessagePassPaiNN, self).__init__(aggr='add')

        self.lin1 = Linear(num_feat, out_channels)
        self.lin2 = Linear(out_channels, 3*out_channels)
        self.lin_rbf = Linear(n_rbf, 3*out_channels)
        self.silu = Func.silu

        #self.prepare = Prepare_Message_Vector(num_nodes)
        self.RBF = BesselBasis(cut_off, n_rbf)
        self.f_cut = CosineCutoff(cut_off)
        self.num_nodes = num_nodes

    def forward(self, s,v, edge_index, edge_attr):

        s = s.flatten(-1)
        v = v.flatten(-2)

        flat_shape_v = v.shape[-1]
        flat_shape_s = s.shape[-1]

        x =torch.cat([s, v], dim = -1)


        x = self.propagate(edge_index, x=x, edge_attr=edge_attr
                            ,flat_shape_s=flat_shape_s, flat_shape_v=flat_shape_v)

        return x

    def message(self, x_j, edge_attr, flat_shape_s, flat_shape_v):


        # Split Input into s_j and v_j
        s_j, v_j = torch.split(x_j, [flat_shape_s, flat_shape_v], dim=-1)

        # r_ij channel
        rbf = self.RBF(edge_attr)
        ch1 = self.lin_rbf(rbf)
        cut = self.f_cut(edge_attr.norm(dim=-1))
        W = torch.einsum('ij,i->ij',ch1, cut) # ch1 * f_cut

        # s_j channel
        phi = self.lin1(s_j)
        phi = self.silu(phi)
        phi = self.lin2(phi)

        # Split
        left, dsm, right = torch.tensor_split(phi*W,3,dim=-1)

        # v_j channel
        normalized = Func.normalize(edge_attr, p=2, dim=1)

        v_j = v_j.reshape(-1, int(flat_shape_v/3), 3)
        hadamard_right = torch.einsum('ij,ik->ijk',right, normalized)
        hadamard_left = torch.einsum('ijk,ij->ijk',v_j,left)
        dvm = hadamard_left + hadamard_right

        # Prepare vector for update
        x_j = torch.cat((dsm,dvm.flatten(-2)), dim=-1)

        return x_j

    def update(self, out_aggr,flat_shape_s, flat_shape_v):

        s_j, v_j = torch.split(out_aggr, [flat_shape_s, flat_shape_v], dim=-1)

        return s_j, v_j.reshape(-1, int(flat_shape_v/3), 3)
class MessagePassPaiNN_NE(MessagePassing):
    def __init__(self, num_feat, out_channels, num_nodes, cut_off=5.0, n_rbf=20):
        super(MessagePassPaiNN_NE, self).__init__(aggr="add")

        self.lin1 = Linear(num_feat, out_channels)
        self.lin2 = Linear(out_channels, 3 * out_channels)
        self.lin_rbf = Linear(n_rbf, 3 * out_channels)
        self.silu = Func.silu

        # self.prepare = Prepare_Message_Vector(num_nodes)
        self.RBF = BesselBasis(cut_off, n_rbf)
        self.f_cut = CosineCutoff(cut_off)
        self.num_nodes = num_nodes
        self.num_feat = num_feat

    def forward(self, s, v, s_nuc, v_nuc, edge_index, edge_attr):

        s = s.flatten(-1)
        v = v.flatten(-2)

        s_nuc = s_nuc.flatten(-1)
        v_nuc = v_nuc.flatten(-2)

        flat_shape_v = v.shape[-1]
        flat_shape_s = s.shape[-1]

        n_nuc = s_nuc.shape[0]
        n_elec = s.shape[0]

        x_p = torch.cat([s_nuc, v_nuc], dim=-1)  # nuclei
        x = torch.cat([s, v], dim=-1)  # electrons

        x = self.propagate(
            edge_index,
            x=(x_p, x),
            edge_attr=edge_attr,
            flat_shape_s=flat_shape_s,
            flat_shape_v=flat_shape_v,
            size=(n_nuc, n_elec),
        )

        return x

    def message(self, x_j, edge_attr, flat_shape_s, flat_shape_v):

        # Split Input into s_j and v_j
        s_j, v_j = torch.split(x_j, [flat_shape_s, flat_shape_v], dim=-1)
        # _, v_i = torch.split(x_i, [flat_shape_s, flat_shape_v], dim=-1)

        # r_ij channel
        rbf = self.RBF(edge_attr)
        ch1 = self.lin_rbf(rbf)
        cut = self.f_cut(edge_attr.norm(dim=-1))
        W = torch.einsum("ij,i->ij", ch1, cut)  # ch1 * f_cut

        # s_j channel
        phi = self.lin1(s_j)
        phi = self.silu(phi)
        phi = self.lin2(phi)

        # Split
        left, dsm, right = torch.split(phi * W, self.num_feat, dim=-1)

        # v_j channel
        normalized = Func.normalize(edge_attr, p=2, dim=1)

        v_j = v_j.reshape(-1, int(flat_shape_v / 3), 3)
        # v_i = v_i.reshape(-1, int(flat_shape_v/3), 3)
        # print(v_j - v_i)
        hadamard_right = torch.einsum("ij,ik->ijk", right, normalized)
        hadamard_left = torch.einsum("ijk,ij->ijk", v_j, left)
        dvm = hadamard_left + hadamard_right

        # Prepare vector for update
        x_j = torch.cat((dsm, dvm.flatten(-2)), dim=-1)

        return x_j

    def update(self, out_aggr, flat_shape_s, flat_shape_v):

        s_j, v_j = torch.split(out_aggr, [flat_shape_s, flat_shape_v], dim=-1)

        return s_j, v_j.reshape(-1, int(flat_shape_v / 3), 3)


In [36]:
class UpdatePaiNN(torch.nn.Module):
    def __init__(self, num_feat, out_channels, num_nodes):
        super(UpdatePaiNN, self).__init__()

        self.lin_up = Linear(2*num_feat, out_channels)
        self.denseU = Linear(num_feat,out_channels, bias = False)
        self.denseV = Linear(num_feat,out_channels, bias = False)
        self.lin2 = Linear(out_channels, 3*out_channels)
        self.silu = Func.silu


    def forward(self, s,v):

        # split and take linear combinations
        #s, v = torch.split(out_aggr, [flat_shape_s, flat_shape_v], dim=-1)

        s = s.flatten(-1)
        v = v.flatten(-2)

        flat_shape_v = v.shape[-1]
        flat_shape_s = s.shape[-1]

        v_u = v.reshape(-1, int(flat_shape_v/3), 3)
        v_ut = torch.transpose(v_u,1,2)
        U = torch.transpose(self.denseU(v_ut),1,2)
        V = torch.transpose(self.denseV(v_ut),1,2)


        # form the dot product
        UV =  torch.einsum('ijk,ijk->ij',U,V)

        # s_j channel
        nV = torch.norm(V, dim=-1)

        s_u = torch.cat([s, nV], dim=-1)
        s_u = self.lin_up(s_u)
        s_u = Func.silu(s_u)
        s_u = self.lin2(s_u)
        #s_u = Func.silu(s_u)

        # final split
        top, middle, bottom = torch.tensor_split(s_u,3,dim=-1)

        # outputs
        dvu = torch.einsum('ijk,ij->ijk',v_u,top)
        dsu = middle*UV + bottom

        #update = torch.cat((dsu,dvu.flatten(-2)), dim=-1)

        return dsu, dvu.reshape(-1, int(flat_shape_v/3), 3)


In [37]:
class PaiNN(torch.nn.Module):
    def __init__(self, num_feat, out_channels, num_nodes, cut_off=5.0, n_rbf=20, num_interactions=3):
        super(PaiNN, self).__init__()
        '''PyG implementation of PaiNN network of Schütt et. al. Supports two arrays
           stored at the nodes of shape (num_nodes,num_feat,1) and (num_nodes, num_feat,3). For this
           representation to be compatible with PyG, the arrays are flattened and concatenated.
           Important to note is that the out_channels must match number of features'''

        self.num_nodes = num_nodes
        self.num_interactions = num_interactions
        self.cut_off = cut_off
        self.n_rbf = n_rbf
        self.linear = Linear(num_feat,num_feat)
        self.silu = Func.silu

        self.list_message = nn.ModuleList(
            [
                MessagePassPaiNN(num_feat, out_channels, num_nodes, cut_off, n_rbf)
                for _ in range(self.num_interactions)
            ]
        )
        self.list_update = nn.ModuleList(
            [
                UpdatePaiNN(num_feat, out_channels, num_nodes)
                for _ in range(self.num_interactions)
            ]
        )


    def forward(self, s,v, edge_index, edge_attr):


        for i in range(self.num_interactions):

            s_temp,v_temp = self.list_message[i](s,v, edge_index, edge_attr)
            s, v = s_temp+s, v_temp+v
            s_temp,v_temp = self.list_update[i](s,v)
            s, v = s_temp+s, v_temp+v

        s = self.linear(s)
        s = self.silu(s)
        s = self.linear(s)

        return v


In [38]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as Func
from torch.nn import Linear

# from message import MessagePassPaiNN, MessagePassPaiNN_NE
# from update import UpdatePaiNN


class PaiNN(torch.nn.Module):
    def __init__(
        self,
        num_feat,
        out_channels,
        num_nodes,
        cut_off=5.0,
        n_rbf=20,
        num_interactions=3,
    ):
        super(PaiNN, self).__init__()
        """PyG implementation of PaiNN network of Schütt et. al. Supports two arrays
           stored at the nodes of shape (num_nodes,num_feat,1) and (num_nodes, num_feat,3). For this
           representation to be compatible with PyG, the arrays are flattened and concatenated.
           Important to note is that the out_channels must match number of features"""

        self.num_interactions = num_interactions
        self.cut_off = cut_off
        self.n_rbf = n_rbf
        self.num_nodes = num_nodes
        self.num_feat = num_feat
        self.out_channels = out_channels
        self.lin = Linear(num_feat, num_feat)
        self.silu = Func.silu

        self.list_message = nn.ModuleList(
            [
                MessagePassPaiNN(num_feat, out_channels, num_nodes, cut_off, n_rbf)
                for _ in range(self.num_interactions)
            ]
        )
        self.list_update = nn.ModuleList(
            [
                UpdatePaiNN(num_feat, out_channels, num_nodes)
                for _ in range(self.num_interactions)
            ]
        )

    def forward(self, s, v, edge_index, edge_attr):

        for i in range(self.num_interactions):

            s_temp, v_temp = self.list_message[i](s, v, edge_index, edge_attr)
            s, v = s_temp + s, v_temp + v
            s_temp, v_temp = self.list_update[i](s, v)
            s, v = s_temp + s, v_temp + v

        s = self.lin(s)
        s = self.silu(s)
        s = self.lin(s)

        return s


class PaiNNElecNuc(torch.nn.Module):
    def __init__(
        self,
        num_feat,
        out_channels,
        num_nodes,
        cut_off=5.0,
        n_rbf=20,
        num_interactions=3,
    ):
        super(PaiNNElecNuc, self).__init__()
        """PyG implementation of PaiNN network of Schütt et. al. Supports two arrays
           stored at the nodes of shape (num_nodes,num_feat,1) and (num_nodes, num_feat,3). For this
           representation to be compatible with PyG, the arrays are flattened and concatenated.
           Important to note is that the out_channels must match number of features"""

        self.num_nodes = num_nodes
        self.num_interactions = num_interactions
        self.cut_off = cut_off
        self.n_rbf = n_rbf
        self.linear = Linear(num_feat, num_feat)
        self.silu = Func.silu

        self.list_message = nn.ModuleList(
            [
                MessagePassPaiNN(num_feat, out_channels, num_nodes, cut_off, n_rbf)
                for _ in range(self.num_interactions)
            ]
        )
        self.list_update = nn.ModuleList(
            [
                UpdatePaiNN(num_feat, out_channels, num_nodes)
                for _ in range(self.num_interactions)
            ]
        )

        self.list_message_NE = nn.ModuleList(
            [
                MessagePassPaiNN_NE(num_feat, out_channels, num_nodes)
                for _ in range(self.num_interactions)
            ]
        )

    def forward(
        self, s, v, s_nuc, v_nuc, edge_index, edge_attr, edge_index_nuc, edge_attr_nuc
    ):

        for i in range(self.num_interactions):

            s_temp, v_temp = self.list_message[i](s, v, edge_index, edge_attr)
            s_temp_NE, v_temp_NE = self.list_message_NE[i](
                s, v, s_nuc, v_nuc, edge_index_nuc, edge_attr_nuc
            )

            s, v = s_temp + s + s_temp_NE, v_temp + v + v_temp_NE
            s_temp, v_temp = self.list_update[i](s, v)
            s, v = s_temp + s, v_temp + v

        s = self.linear(s)
        s = self.silu(s)
        s = self.linear(s)

        return s, v


# Undersøg Data

RuntimeError: Tensors must have same number of dimensions: got 2 and 1